# Tests of Buffered Iterator of Buffered Iterator

### Imports

In [1]:
from pprint import pprint
import random
from buffered_iterator import BufferedIterator, BufferedIteratorEOF
from buffered_iterator import BufferedIteratorValueError

from sections import SectionBreak, Section
from sections import Rule, RuleSet, ProcessingMethods


### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Line Count Tests')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)


### Display Functions

In [3]:

def buffered_iterator_compare(iter1, iter2=None, iter3=None, 
                              label1='From Iterator', 
                              label2='To Iterator', label3=''):
    
    def extract_attrs(buf_obj, requested_item, as_list=True):
        if not buf_obj:
            text = ''
        elif as_list:
            text = str(list(buf_obj.__getattribute__(requested_item)))
        else:
            text = str(buf_obj.__getattribute__(requested_item))
        return text
        
    def extract_attr_text(requested_item, iter1, iter2=None, iter3=None, 
                        as_list=True):    
        attr_text = {
            1: extract_attrs(iter1, requested_item, as_list),
            2: extract_attrs(iter2, requested_item, as_list),
            3: extract_attrs(iter3, requested_item, as_list),
        }
        return attr_text


    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<35s}',
        '{second_iter_item:<35s}',
        '{third_iter_item:<35s}\n'
        ])   
    attr_group = {
        'Previous Items': ('previous_items', True),
        'Future Items': ('future_items', True),
        'Item Count': ('item_count', False),
        'Step Back': ('_step_back', False),
        'Buffer Size': ('buffer_size', False)
        }

    row_list = [
        row_template.format(
            Label='',
            first_iter_item=label1, 
            second_iter_item=label2, 
            third_iter_item=label3)
                ]

    for label, attr_s in attr_group.items():
        requested_item, as_list = attr_s
        text_group = extract_attr_text(requested_item, iter1, iter2, iter3, as_list)
        text_line = row_template.format(Label=label, 
                        first_iter_item=text_group[1],
                        second_iter_item=text_group[2],
                        third_iter_item=text_group[3])
        row_list.append(text_line)
    
    iterator_compare_str = ''.join(row_list)
    
    return iterator_compare_str

# Nested Buffered Iterators

Create three levels of nested BufferedIterators to observe the buffer spaces.
1. BufferedIterator of String integers from '0' to `num_items`
2. BufferedIterator of above (1.) BufferedIterator
3. BufferedIterator of Section.process of above (2.) BufferedIterator

**Section Definition**
- The section is defined with start and end points that will leave an item in 
  the Future Items buffer.  
- The section also has a processor that consumes two source items for each 
  section item.
- Buffer Size is set to be larger than the source so all activity will be 
  captured.

```
def pairs(source):
    for item in source:
        yield tuple([item, next(source)])

section_buf = Section(
    section_name='Section for testing Nested Buffered Iterators',
    start_section=SectionBreak('1', break_offset='After'),
    end_section=SectionBreak('6', break_offset='Before'),
    processor=[pairs]
    )
```

In [4]:
buffer_size = 20
num_items = 15

str_source = BufferedIterator((str(i) for i in range(num_items)), 
                              buffer_size=buffer_size)

top_source = BufferedIterator(str_source, buffer_size=buffer_size)

def pairs(source):
    for item in source:
        yield tuple([item, next(source)])

section_buf = Section(
    section_name='Section for testing Nested Buffered Iterators',
    start_section=SectionBreak('1', break_offset='After'),
    end_section=SectionBreak('6', break_offset='Before'),
    processor=[pairs]
    )

sec_source = BufferedIterator(section_buf.process(top_source))

# Run the iterator
[i for i in sec_source]
   

print(buffered_iterator_compare(str_source, top_source, sec_source, 
                                label1='str_source', 
                                label2='top_source',
                                label3='sec_source'))


print(f'Item Count: {section_buf.item_count}  \t'
        f'Source Item Count {section_buf.source.item_count}  \t'
        f'Section Source Index: {section_buf.source_item_count}')

section_buf.source_index

	                    str_source                         top_source                         sec_source                         
	Previous Items      ['0', '1', '2', '3', '4', '5', '6']['0', '1', '2', '3', '4', '5', '6'][('2', '3'), ('4', '5')]           
	Future Items        []                                 []                                 []                                 
	Item Count          7                                  7                                  2                                  
	Step Back           0                                  0                                  0                                  
	Buffer Size         20                                 20                                 5                                  

Item Count: 2  	Source Item Count 6  	Section Source Index: 6


[4, 6]

## Tests

In [5]:
test_text = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

test_section = Section(
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )
        

test_iter = BufferedIterator(test_text)

print(test_section.read(test_iter))
print(buffered_iterator_compare(test_iter, label1='After first Read', label2=''))
print(f'Item Count: {test_section.item_count}  \t'
        f'Source Item Count {test_section.source.item_count}  \t'
        f'Section Source Index: {test_section.source_item_count}')


print(test_section.read(test_iter))
print(buffered_iterator_compare(test_iter, label1='After second Read', label2=''))
print(f'Item Count: {test_section.item_count}  \t'
        f'Source Item Count {test_section.source.item_count}  \t'
        f'Section Source Index: {test_section.source_item_count}')


print(test_section.read(test_iter))
print(buffered_iterator_compare(test_iter, label1='After second Read', label2=''))
print(f'Item Count: {test_section.item_count}  \t'
        f'Source Item Count {test_section.source.item_count}  \t'
        f'Section Source Index: {test_section.source_item_count}')


['StartSection Name: A', 'EndSection Name: A']
	                    After first Read                                                                                         
	Previous Items      ['Text to be ignored', 'StartSection Name: A', 'EndSection Name: A']                                                                      
	Future Items        ['StartSection Name: B']                                                                                 
	Item Count          3                                                                                                        
	Step Back           0                                                                                                        
	Buffer Size         5                                                                                                        

Item Count: 2  	Source Item Count 3  	Section Source Index: 3
['StartSection Name: B', 'EndSection Name: B']
	                    After second Read          

In [19]:
test_text = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

test_section = Section(
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )
        

test_iter = BufferedIterator(test_text)

print(test_section.read(test_iter))
print(test_section.read(test_iter))
print(test_section.read(test_iter))


['StartSection Name: A', 'EndSection Name: A']
['StartSection Name: B', 'EndSection Name: B']
[]


In [6]:
test_text = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )

full_section = Section(
    subsections=sub_section,
    keep_partial=False
    )


In [7]:
a = full_section.read(test_text)


In [8]:
a

[['StartSection Name: A', 'EndSection Name: A']]

In [9]:

b = full_section.read(test_text)

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
        )
print(full_section.read(test_text))

print(full_section.read(test_text))


[['StartSection Name: A', 'EndSection Name: A']]
[['StartSection Name: A', 'EndSection Name: A']]


In [10]:
a

[['StartSection Name: A', 'EndSection Name: A']]

### Validate minimum bluffer size is 1

In [11]:
try:
    test_iter = BufferedIterator(range(5), buffer_size=0)
except BufferedIteratorValueError: 
    print('pass')

pass


### Basic Parameters

In [12]:
buffer_size = 8
num_items = 15

		
### Make BufferedIterator of BufferedIterator

- Verify that:
- step back only impacts top iterator
- queues in both store items from next
- future Items in top are pulled by next without calling inner iterator
- update works to pass future items from top to inner 
- What happens when updating top iterator with sub iterator?

In [13]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_top =   5 # random.randint(2, buffer_size_top-1)
back_count_top =  2 # random.randint(1, fwd_count_top)

for i in range(fwd_count_top):
    next(top_iter)
top_iter.backup(back_count_top)
print(buffered_iterator_compare(top_iter, sub_iter))

fwd_count_sub = 3
back_count_sub = 1

for i in range(fwd_count_sub):
    next(sub_iter)
sub_iter.backup(back_count_sub)

print(buffered_iterator_compare(top_iter, sub_iter))

	                    From Iterator                      To Iterator                                                           
	Previous Items      [0, 1, 2]                          []                                                                    
	Future Items        [3, 4]                             []                                                                    
	Item Count          3                                  0                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     

	                    From Iterator                      To Iterator                                                           
	Previous Items      [1, 2, 3, 4, 5]                    [3, 4]                                                

#### Test that queues in both store items from next
- Advance Sub Iterator by Top Iterator Buffer Size and back by smaller amount
- Top Iterator Previous Items should contain all items
- Sub Iterator should split items between Previous Items and Future Items.
- Top Iterator Item Count should equal Buffer Size.
- Sub Iterator Item Count should equal Buffer Size - Step Back.

In [14]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub = buffer_size_top
back_count_sub = random.randint(1, buffer_size_top-1)

for i in range(fwd_count_sub):
    next(sub_iter)
sub_iter.backup(back_count_sub)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(top_iter.previous_items) == buffer_size_top)
print(top_iter.previous_items == sub_iter.previous_items + sub_iter.future_items)

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2, 3, 4]                    [0, 1]                                                                
	Future Items        []                                 [2, 3, 4]                                                             
	Item Count          5                                  2                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     

True
True


#### Test that step back in top only impacts top iterator
- Advance Sub Iterator by Top Iterator Buffer Size.
- Backup Top Iterator by smaller amount.
- Top Iterator should split items between Previous Items and Future Items.
- Sub Iterator Previous Items should contain all items.
- Top Iterator Item Count should equal Buffer Size - Step Back.
- Sub Iterator Item Count should equal Buffer Size.

In [15]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   buffer_size_top
back_count_top =  random.randint(1, fwd_count_sub)

for i in range(fwd_count_sub):
    next(sub_iter)   
top_iter.backup(back_count_top)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1]                             [0, 1, 2, 3, 4]                                                       
	Future Items        [2, 3, 4]                          []                                                                    
	Item Count          2                                  5                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     

True
True


#### Test that future Items in top are pulled by next without calling inner iterator
- Advance Sub Iterator by Top Iterator Buffer Size.
- Backup Top Iterator by smaller amount.
- Advance Top Iterator by even smaller amount.
- Top Iterator should split items between Previous Items and Future Items.
- Sub Iterator Previous Items should contain all items.
- Top Iterator Item Count should equal Buffer Size - Step Back.
- Sub Iterator Item Count should equal Buffer Size.

In [16]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   buffer_size_top
back_count_top =  random.randint(1, fwd_count_sub)
fwd_count_top =   random.randint(1, back_count_top)

for i in range(fwd_count_sub):
    next(sub_iter)   
top_iter.backup(back_count_top)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))
for i in range(fwd_count_top):
    next(top_iter)   
 
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))
   
print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2, 3]                       [0, 1, 2, 3, 4]                                                       
	Future Items        [4]                                []                                                                    
	Item Count          4                                  5                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2, 3, 4]                    [0, 1, 2, 3, 4]                                       

#### Test that link works to pass previous items from top to sub 
- Advance Sub Iterator by Top Iterator Buffer Size and back by smaller amount
- Top Iterator Previous Items should contain all items
- Sub Iterator should split items between Previous Items and Future Items.
- Top Iterator Item Count should equal Buffer Size.
- Sub Iterator Item Count should equal Buffer Size - Step Back.

In [17]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   random.randint(1, buffer_size_sub - 1)
fwd_count_top =   random.randint(1, buffer_size_top - fwd_count_sub)
back_count_top =  random.randint(1, fwd_count_top)

for i in range(fwd_count_sub):
    next(sub_iter)   

for i in range(fwd_count_top):
    next(top_iter) 

top_iter.backup(back_count_top)
next(sub_iter)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))


sub_iter.link(top_iter)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)
print(list(sub_iter))

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2]                          [0, 1, 2]                                                             
	Future Items        [3]                                []                                                                    
	Item Count          3                                  3                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     

	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2]                          [0, 1, 2]                                             

### Mixing Sub and top steps can loose items or duplicate

In [18]:
num_items = 10
buffer_size_top = 10
buffer_size_sub = 10

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub = buffer_size_sub - 5
fwd_count_top = buffer_size_top - fwd_count_sub - 3
back_count_top = 4
fwd_count_sub2 = 2

print(f'Advance Sub Iterator by {fwd_count_sub} (Both iterators advance)')
for i in range(fwd_count_sub):
    next(sub_iter)   
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Advance Top Iterator by {fwd_count_top} (Sub Iterator does not advance).')
for i in range(fwd_count_top):
    next(top_iter) 
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Backup Top Iterator by {back_count_top} (Sub Iterator does not change).')
top_iter.backup(back_count_top)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Advance Sub Iterator by {fwd_count_sub2} (Sub Iterator gets previous items from top iterator).')
for i in range(fwd_count_sub2):
    next(sub_iter)     
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

Advance Sub Iterator by 5 (Both iterators advance)
	                    Top Iterator                       Sub Iterator                                                          
	Previous Items      [0, 1, 2, 3, 4]                    [0, 1, 2, 3, 4]                                                       
	Future Items        []                                 []                                                                    
	Item Count          5                                  5                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         10                                 10                                                                    

Advance Top Iterator by 2 (Sub Iterator does not advance).
	                    Top Iterator                       Sub Iterator                                                          
